<img src="https://raw.githubusercontent.com/PinkWink/DataScience/master/img/title.jpg" width="200" height="200"><br>

# 4장 - 셀프 주유소는 정말 저렴할까



이번 장에서는 아주 간단한 질문에 답하는 것을 함꼐 해보려 한다. 누군가 셀프 주유소는 정말 저렴한가? 에 대해 물었다면 데이터를 만지는 사람들은 어떻게 할까? 답은 바로 주유소의 가격을 조사해서 셀프 주유소와 아닌 주유소를 구분해서 비교하면 된다.
앞서 우리는 BeautifulSoup를 사용했다. 그걸로도 많은 일을 할 수 있다. 하지만 몇가지 문제로 인해 BeautifulSoup만으로는 접근할 수 없는 인터넷 정보가 있다. 그래서 사용하는 것이 바로 Selenium 이다.
우선 주유소의 가격을 비교하는 정부 사이트인 Opninet에 접속해서 정보를 모아야 한다.

In [25]:
from selenium import webdriver

먼저 selenium 에서 webdriver를 import 한다. 그리고 네이버(www.naver.com)에 접속해 보자.

In [26]:
driver = webdriver.Chrome('C:/Users/이재윤/Downloads/chromedriver_win32/chromedriver')
driver.get("http://naver.com")

위처럼 네이버에 접속을 하면 'chrome이 자동화된 테스트 소프트웨어에 의해 제어되고 있습니다' 라는 문구와 함께 웹 브라우저가 또 하나 떠 있을 텐데, 저 웹 브라우저는 우리가 코드로 움직일 브라우저이다. 그래서 가급적 우리가 생성한 크롬 브라우저는 손으로 조작하면 안된다. 코드를 작성할 때 혼선이 생길 수 있다. 별도로 네이버창을 하나 더 열어서 작업하자.

In [29]:
driver.save_screenshot('C:/Users/이재윤/images/001.jpg')

True

selenium은 save_screenshot 명령으로 화면을 캡처할 수 있다.

In [38]:
elem_login = driver.find_element_by_id("id")
elem_login.clear()
elem_login.send_keys("wodbsdbsk")

elem_login = driver.find_element_by_id("pw")
elem_login.clear()
elem_login.send_keys("sorkwodbs1!")

네이버에 로그인 정보를 입력하는 곳이 있다. 크롬 드라이버로 네이버에 로그인하고 싶다면 당연히 id와 비밀번호를 입력해야 한다. 개발자 도구를 이용해서 id와 비밀번호를 입력하는 부분의 html 소스 코드를 확인해 보면 id= 라는 항목에 id 혹은 pw 라고 되어있다. selenium이 제공하는 명령 중 find_element_by_id 를 이용해서 id와 pw를 찾으면 된다. 위 코드를 치면 로그인 정보가 입력되어 있을 것이다.

그런 후에 로그인 개발자 도구로 로그인 버튼 쪽을 클릭하면 특정 코드가 하이라이트 되어 있을 것이다. 그곳에서 마우스 오른쪽 버튼을 클릭 후 copy 항목으로 가서 copy xpath를 선택하면 로그인 버튼의 xpath를 복사할 수 있다.


In [39]:
xpath = '''//*[@id="frmNIDLogin"]/fieldset/input'''
driver.find_element_by_xpath(xpath).click()

그리고 위 코드를 작성하면서 방금 복사한 xpath를 xpath =  뒤에 붙여 넣기 하면 된다.
이렇게 로그인을 한 후 메일(mail.naver.com)에 접근해 보자.

In [40]:
driver.get("http://mail.naver.com")

원하는 곳으로 이동 했다면 Beautiful Soup를 이용해서 페이지 내용을 읽어오게 된다.

In [48]:
from bs4 import BeautifulSoup

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

driver.page_source를 사용하면 현재 selenium이 접근한 페이지의 소스를 넘겨 받을 수 있다.

이제 크롬 개발자 도구를 이용해서 메일을 보낸 사람이 나타나는 곳의 태그를 확인해두자.

In [50]:
raw_list = soup.find_all('div', 'name _ccr(lst.from)')
raw_list

[<div class="name _ccr(lst.from)"><span class="blind">보낸 이:</span><a class="_c1(myContextMenu|showSenderContextLayer|list|4) _stopDefault" href="#" title='"네이버메일" &lt;navermail_noreply@navercorp.com&gt;'>네이버메일</a></div>,
 <div class="name _ccr(lst.from)"><span class="blind">보낸 이:</span><a class="_c1(myContextMenu|showSenderContextLayer|list|3) _stopDefault" href="#" title="&lt;no_reply@its-lil.com&gt;">no_reply@its-lil.com</a></div>,
 <div class="name _ccr(lst.from)"><span class="blind">보낸 이:</span><a class="_c1(myContextMenu|showSenderContextLayer|list|2) _stopDefault" href="#" title='"네이버" &lt;webmaster@naver.com&gt;'>네이버</a></div>,
 <div class="name _ccr(lst.from)"><span class="blind">보낸 이:</span><a class="_c1(myContextMenu|showSenderContextLayer|list|1) _stopDefault" href="#" title='"네이버" &lt;webmaster@naver.com&gt;'>네이버</a></div>]

위처럼 find_all 명령을 사용하면 된다.

In [51]:
send_list = [raw_list[n].find('a').get_text() for n in range(0, len(raw_list))]
send_list

['네이버메일', 'no_reply@its-lil.com', '네이버', '네이버']

손쉽게 메일을 보낸 사람의 리스트를 확보할 수 있다. 이제 크롬 드라이버를 닫아야 한다.

In [52]:
driver.close()

close()명령을 이용해서 실행된 크롬 드라이버를 종료할 수 있다.

## 4-2 서울시 구별 주유소 가격 정보 얻기


앞에서 배운 selenium 지식으로 https://goo.gl/VH1A5t 에 접속해서 서울시 구별 주유소 정보를 받아오자.



In [53]:
from selenium import webdriver

In [55]:
driver = webdriver.Chrome('C:/Users/이재윤/Downloads/chromedriver_win32/chromedriver')
driver.get('http://www.naver.com')

In [57]:
driver.get('http://www.opinet.co.kr/searRgSelect.do')

우리는 서울시만 검색할 것이니 가만히 두고, 처음에 나타나는 구에 종로구 라는 글자가 있는 부분은 바꿔줘야 한다. 
리스트 박스 형태로 되어 있어서 해당 리스트의 내용을 받아와서 순차적으로 반환해주면 된다.

In [59]:
gu_list_raw = driver.find_element_by_xpath('''//*[@id="SIGUNGU_NM0"]''')
gu_list = gu_list_raw.find_elements_by_tag_name('option')

크롬 개발자 도구를 이용해서롬 개발자 도구를 이용해서 구 이름이 위치하는 곳을 클릭하면 나타나는 우측 코드에 마우스 오른쪽 버튼으로 Xpath를 복사하기를 하면 된다. 그렇게 확보한 xpath를 이용해서 element를 찾고 gu_list_raw 변수에 저장한다.
구 리스트는 find_elements_by_tag_name으로 option이라는 태그를 찾으면 된다.

In [60]:
gu_names = [option.get_attribute("value") for option in gu_list]
gu_names.remove('')
gu_names

['강남구',
 '강동구',
 '강북구',
 '강서구',
 '관악구',
 '광진구',
 '구로구',
 '금천구',
 '노원구',
 '도봉구',
 '동대문구',
 '동작구',
 '마포구',
 '서대문구',
 '서초구',
 '성동구',
 '성북구',
 '송파구',
 '양천구',
 '영등포구',
 '용산구',
 '은평구',
 '종로구',
 '중구',
 '중랑구']

그렇게 얻은 결과가 나타난다. 구 이름을 전체적으로 다 알았다. 구 이름이 있는 태그에 위 코드에서 저장한 gu_names에서 첫 번째 것을 한번 시험 삼아 입력하자.

In [63]:
element = driver.find_element_by_id("SIGUNGU_NM0")
element.send_keys(gu_names[0])

In [65]:
driver.save_screenshot('C:/Users/이재윤/images/002.jpg')

True

그러면 구 이름부분이 변경된 것을 알 수 있다. 그리고 조회 버튼을 누르면 된다. 조회 버튼의 xpath도 알아낼 수 있다.

In [68]:
xpath ='''//*[@id="searRgSelect"]/span'''
element_sel_gu = driver.find_element_by_xpath(xpath).click()

그리고 해당 xpath를 찾아서 click()을 붙여 주면 된다. 결과가 나타난 곳 아래에 엑셀저장 버튼을 눌러서 엑셀로 내용을 저장해야 한다. 지금까지 한 것처럼 xpat를 알아내서 엑셀 저장 버튼을 누른다.

In [75]:
xpath = """//*[@id="glopopd_excel"]/span"""
element_get_excel = driver.find_element_by_xpath(xpath).click()

이렇게 엑셀로 저장 버튼까지 누르고 나면 당연히 크럼 드라이버가 실행하는 브라우저가 지정된 다운로드 폴더에 파일을 다운로드 한다. 

In [79]:
import time
from tqdm import tqdm_notebook

for gu in tqdm_notebook(gu_names):
    element = driver.find_element_by_id('SIGUNGU_NM0')
    element.send_keys(gu)
    
    time.sleep(2)
    
    xpath ='''//*[@id="searRgSelect"]/span'''
    element_sel_gu = driver.find_element_by_xpath(xpath).click()
    
    time.sleep(1)
    
    xpath = '''//*[@id="glopopd_excel"]/span'''
    element_get_excel = driver.find_element_by_xpath(xpath).click()
    
    time.sleep(1)


<ipython-input-79-da81b98703dc>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for gu in tqdm_notebook(gu_names):


  0%|          | 0/25 [00:00<?, ?it/s]

이제 서울시 25개 구에 대해 반복문을 수행하면 된다.적절히 중간중간에 기다리라는 time.sleep 명령을 사용했고 25개 구에 대한 주유소 기름값을 저장한 엑셀파일이 다운로드 폴더에 저장된다.

In [81]:
driver.close()

## 4-3 구별 주유 가격에 대한 데이터의 정리


앞서 받은 25개의 엑셀 파일을 우리가 다루는 data 폴더로 옮긴다. 이전에 배운 엑셀 파일을 read하는 명령으로 하나하나 읽으면 25줄을 입력해야 하지만 파이썬에는 이를 해결해줄 좋은 모듈이 있다.
pandas와 파일 glob 라고 하는 파일 경로등을 쉽게 접근할 수 있게 해주는 모듈을 import 하자.

In [82]:
import pandas as pd
from glob import glob

In [83]:
glob('C:/Users/이재윤/DataScience/data/지역*xls')

['C:/Users/이재윤/DataScience/data\\지역_위치별(주유소) (1).xls',
 'C:/Users/이재윤/DataScience/data\\지역_위치별(주유소) (10).xls',
 'C:/Users/이재윤/DataScience/data\\지역_위치별(주유소) (11).xls',
 'C:/Users/이재윤/DataScience/data\\지역_위치별(주유소) (12).xls',
 'C:/Users/이재윤/DataScience/data\\지역_위치별(주유소) (13).xls',
 'C:/Users/이재윤/DataScience/data\\지역_위치별(주유소) (14).xls',
 'C:/Users/이재윤/DataScience/data\\지역_위치별(주유소) (15).xls',
 'C:/Users/이재윤/DataScience/data\\지역_위치별(주유소) (16).xls',
 'C:/Users/이재윤/DataScience/data\\지역_위치별(주유소) (17).xls',
 'C:/Users/이재윤/DataScience/data\\지역_위치별(주유소) (18).xls',
 'C:/Users/이재윤/DataScience/data\\지역_위치별(주유소) (19).xls',
 'C:/Users/이재윤/DataScience/data\\지역_위치별(주유소) (2).xls',
 'C:/Users/이재윤/DataScience/data\\지역_위치별(주유소) (20).xls',
 'C:/Users/이재윤/DataScience/data\\지역_위치별(주유소) (21).xls',
 'C:/Users/이재윤/DataScience/data\\지역_위치별(주유소) (22).xls',
 'C:/Users/이재윤/DataScience/data\\지역_위치별(주유소) (23).xls',
 'C:/Users/이재윤/DataScience/data\\지역_위치별(주유소) (24).xls',
 'C:/Users/이재윤/DataScience/data\\지역_위치별(주유소) (3).x

위와 같이 ./data 폴더 안에 지역으로 시작하는 xls 파일 전체를 의미하는 ../data/지역*.xls과 같은 명령을 사용할 수 있다.

In [87]:
stations_files = glob('C:/Users/이재윤/DataScience/data/지역*xls')
stations_files

['C:/Users/이재윤/DataScience/data\\지역_위치별(주유소) (1).xls',
 'C:/Users/이재윤/DataScience/data\\지역_위치별(주유소) (10).xls',
 'C:/Users/이재윤/DataScience/data\\지역_위치별(주유소) (11).xls',
 'C:/Users/이재윤/DataScience/data\\지역_위치별(주유소) (12).xls',
 'C:/Users/이재윤/DataScience/data\\지역_위치별(주유소) (13).xls',
 'C:/Users/이재윤/DataScience/data\\지역_위치별(주유소) (14).xls',
 'C:/Users/이재윤/DataScience/data\\지역_위치별(주유소) (15).xls',
 'C:/Users/이재윤/DataScience/data\\지역_위치별(주유소) (16).xls',
 'C:/Users/이재윤/DataScience/data\\지역_위치별(주유소) (17).xls',
 'C:/Users/이재윤/DataScience/data\\지역_위치별(주유소) (18).xls',
 'C:/Users/이재윤/DataScience/data\\지역_위치별(주유소) (19).xls',
 'C:/Users/이재윤/DataScience/data\\지역_위치별(주유소) (2).xls',
 'C:/Users/이재윤/DataScience/data\\지역_위치별(주유소) (20).xls',
 'C:/Users/이재윤/DataScience/data\\지역_위치별(주유소) (21).xls',
 'C:/Users/이재윤/DataScience/data\\지역_위치별(주유소) (22).xls',
 'C:/Users/이재윤/DataScience/data\\지역_위치별(주유소) (23).xls',
 'C:/Users/이재윤/DataScience/data\\지역_위치별(주유소) (24).xls',
 'C:/Users/이재윤/DataScience/data\\지역_위치별(주유소) (3).x

이제 station_files 변수에 각 엑셀 파일의 경로와 이름을 리스트로 저장한다.

In [88]:
tmp_raw = []

for file_name in stations_files:
    tmp = pd.read_excel(file_name, header=2)
    tmp_raw.append(tmp)
    
station_raw = pd.concat(tmp_raw)

그리고 read_excel로 각 파일을 반복문을 이용해서 읽은 후 tml_raw 변수에 append 시킨다. 반복문이 끝나고 나면 concat 명령으로 쉽게 하나로 합칠 수 있다.

In [89]:
station_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 537 entries, 0 to 45
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   지역      537 non-null    object
 1   상호      537 non-null    object
 2   주소      537 non-null    object
 3   상표      537 non-null    object
 4   전화번호    537 non-null    object
 5   셀프여부    537 non-null    object
 6   고급휘발유   537 non-null    object
 7   휘발유     537 non-null    object
 8   경유      537 non-null    object
 9   실내등유    537 non-null    object
dtypes: object(10)
memory usage: 46.1+ KB


총 545개의 주유소 정보가 저장된 것을 알 수 있다. 하지만 가격 정보가 숫자형(int, float)이 아니어서 나중에 처리하자.

In [90]:
station_raw.head()

,지역,상호,주소,상표,전화번호,셀프여부,고급휘발유,휘발유,경유,실내등유
0,서울특별시,오렌지주유소,서울 강동구 성안로 102 (성내동),SK에너지,02-484-6165,N,-,1554,1354,997
1,서울특별시,구천면주유소,서울 강동구 구천면로 357 (암사동),현대오일뱅크,02-441-0536,N,-,1556,1355,-
2,서울특별시,GS칼텍스㈜직영 신월주유소,서울 강동구 양재대로 1323 (성내동),GS칼텍스,02-475-2600,N,1855,1559,1349,1000
3,서울특별시,광성주유소,서울 강동구 올림픽로 673 (천호동),S-OIL,02-470-5133,N,-,1578,1388,1100
4,서울특별시,(주)소모에너지엔테크놀러지성내주유소,서울 강동구 올림픽로 578 (성내동),GS칼텍스,02-479-3838,Y,-,1588,1388,-


In [91]:
stations = pd.DataFrame({'Oil_store': station_raw['상호'],
                         '주소': station_raw['주소'],
                         '가격': station_raw['휘발유'],
                         '셀프': station_raw['셀프여부'],
                         '상표': station_raw['상표']
                        })
stations.head()

,Oil_store,주소,가격,셀프,상표
0,오렌지주유소,서울 강동구 성안로 102 (성내동),1554,N,SK에너지
1,구천면주유소,서울 강동구 구천면로 357 (암사동),1556,N,현대오일뱅크
2,GS칼텍스㈜직영 신월주유소,서울 강동구 양재대로 1323 (성내동),1559,N,GS칼텍스
3,광성주유소,서울 강동구 올림픽로 673 (천호동),1578,N,S-OIL
4,(주)소모에너지엔테크놀러지성내주유소,서울 강동구 올림픽로 578 (성내동),1588,Y,GS칼텍스


위 처럼 원하는 컬럼만 가지고 오고 이름도 다시 정의해서 stations 라는 변수에 저장하자.

여기에 추가로 주소에서 구 이름만 추출한다. 그래서 구별 주유 가격도 조사하자.

In [92]:
stations['구'] = [eachAddress.split()[1] for eachAddress in stations['주소']]
stations.head()

,Oil_store,주소,가격,셀프,상표,구
0,오렌지주유소,서울 강동구 성안로 102 (성내동),1554,N,SK에너지,강동구
1,구천면주유소,서울 강동구 구천면로 357 (암사동),1556,N,현대오일뱅크,강동구
2,GS칼텍스㈜직영 신월주유소,서울 강동구 양재대로 1323 (성내동),1559,N,GS칼텍스,강동구
3,광성주유소,서울 강동구 올림픽로 673 (천호동),1578,N,S-OIL,강동구
4,(주)소모에너지엔테크놀러지성내주유소,서울 강동구 올림픽로 578 (성내동),1588,Y,GS칼텍스,강동구


위 출력을 보면 주소 컬럼의 주소를 봤을 때 빈 칸을 기준으로 분리(split) 시키고 두 번째 단어를 선택하면 구 이름이 될 것 같다.


일단 head()만 조사했을 때는 이상 없어 보이지만 5백여 개나 되는 데이터를 다 보기에는 난감하다. 이때는 unique() 검사를 수행하자.

In [94]:
stations['구'].unique()

array(['강동구', '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '서울특별시', '성북구',
       '송파구', '양천구', '영등포구', '강북구', '용산구', '은평구', '종로구', '중구', '중랑구',
       '강서구', '관악구', '광진구', '구로구', '금천구', '노원구', '도봉구', '특별시', '강남구'],
      dtype=object)

In [95]:
stations[stations['구']=='서울특별시']

,Oil_store,주소,가격,셀프,상표,구
12,SK네트웍스(주)효진주유소,1 서울특별시 성동구 동일로 129 (성수동2가),1654,N,SK에너지,서울특별시


서울 특별시를 확인해보니 애초 주소가 입력될 때 알 수 없는 글자가 하나 들어가서 칸 수가 맞지 않았다.


In [97]:
stations.loc[stations['구']=='서울특별시','구'] = '성동구'
stations['구'].unique()

array(['강동구', '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구',
       '양천구', '영등포구', '강북구', '용산구', '은평구', '종로구', '중구', '중랑구', '강서구',
       '관악구', '광진구', '구로구', '금천구', '노원구', '도봉구', '특별시', '강남구'],
      dtype=object)

위와 같이 직접 변경하자.

In [98]:
stations[stations['구']=='특별시']

,Oil_store,주소,가격,셀프,상표,구
10,서현주유소,서울 특별시 도봉구 방학로 142 (방학동),1524,Y,S-OIL,특별시


또 특별시로 되었던 것도 검색 했다. 이번엔 서울특별시라 적지 않고 서울과 특별시를 띄어쓰기를 해서 발생한 문제이다.

In [101]:
stations.loc[stations['구']=='특별시','구']= '도봉구'
stations['구'].unique()

array(['강동구', '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구',
       '양천구', '영등포구', '강북구', '용산구', '은평구', '종로구', '중구', '중랑구', '강서구',
       '관악구', '광진구', '구로구', '금천구', '노원구', '도봉구', '강남구'], dtype=object)